In [1]:
#| include: false

import os
import pandas as pd
import numpy as np
# import plotly.express as px
import toml
# import psrc_theme
from pathlib import Path
import util

# to show plotly figures in quarto HTML file
# import plotly.io as pio
# pio.renderers.default = "plotly_mimetype+notebook_connected"
# pio.templates.default = "simple_white+psrc_color" # set plotly template

config = toml.load(Path('../../../../configuration')/'summary_configuration.toml')

# read all summary data files
summary_data = util.SummaryData(config)

%store summary_data

Stored 'summary_data' (SummaryData)


In [2]:
# VMT, VHT and delay
df_network = summary_data.network_summary.copy()

In [3]:
# transit boardings
df_daily_boardings_by_agency = pd.read_csv(Path(config['output_path'])/'transit/daily_boardings_by_agency.csv', index_col='agency_name')
# mode share
df_trip = pd.read_csv(Path(config['output_path'])/'agg/dash/mode_share_county.csv')
df_mode = df_trip.groupby('mode').sum()[['trexpfac']]/df_trip['trexpfac'].sum()
# emission
df_emissions_summary = pd.read_csv(Path(config['output_path'])/'emissions/emissions_summary.csv')
df = df_emissions_summary[df_emissions_summary['veh_type'].isin(['light','medium','heavy'])]
df_emissions = df.groupby('pollutant_name').sum()

C:\Users\Modeller\AppData\Local\Temp\ipykernel_25364\4235094857.py:5: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_mode = df_trip.groupby('mode').sum()[['trexpfac']]/df_trip['trexpfac'].sum()
C:\Users\Modeller\AppData\Local\Temp\ipykernel_25364\4235094857.py:9: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_emissions = df.groupby('pollutant_name').sum()


In [9]:
data = {'VMT':               [df_network['VMT'].sum()],
        'VHT':               [df_network['VHT'].sum()],
        'Delay':             [df_network['total_delay'].sum()],
        'Transit Boardings': [df_daily_boardings_by_agency['boardings'].sum()],
        '% Transit':         [df_mode.loc['Transit']['trexpfac']],
        'CO2e':              [df_emissions.loc['CO2 Equivalent']['total_daily_tons']]}

# Create DataFrame
pd.DataFrame(data).style.\
        format('{:,.0f}', subset=['VMT','VHT','Delay','Transit Boardings','CO2e']).\
        format('{:.1%}', subset=['% Transit']).\
        hide(axis="index")


VMT,VHT,Delay,Transit Boardings,% Transit,CO2e
"74,794,942","2,216,555","195,921","480,586",2.0%,"41,543"
